modified from:  
https://medium.com/@curiousily/human-activity-recognition-using-lstms-on-android-tensorflow-for-hackers-part-vi-492da5adef64

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from matplotlib import pyplot as plt

RANDOM_SEED = 42

Using TensorFlow backend.


In [2]:
columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
df = pd.read_csv('WISDM_ar_v1.1_raw.txt', header = None, names = columns)
df = df.dropna()

In [3]:
df

,user,activity,timestamp,x-axis,y-axis,z-axis
0,33,Jogging,49105962326000,-0.694638,12.680544,0.503953
1,33,Jogging,49106062271000,5.012288,11.264028,0.953424
2,33,Jogging,49106112167000,4.903325,10.882658,-0.081722
3,33,Jogging,49106222305000,-0.612916,18.496431,3.023717
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164
5,33,Jogging,49106442306000,1.375655,-2.492524,-6.510526
6,33,Jogging,49106542312000,-0.612916,10.569390,5.706926
7,33,Jogging,49106652389000,-0.503953,13.947236,7.055340
8,33,Jogging,49106762313000,-8.430995,11.413852,5.134871
9,33,Jogging,49106872299000,0.953424,1.375655,1.648062


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1098203 entries, 0 to 1098203
Data columns (total 6 columns):
user         1098203 non-null int64
activity     1098203 non-null object
timestamp    1098203 non-null int64
x-axis       1098203 non-null float64
y-axis       1098203 non-null float64
z-axis       1098203 non-null float64
dtypes: float64(3), int64(2), object(1)
memory usage: 58.7+ MB


In [17]:
df['activity'].unique()

array(['Jogging', 'Walking', 'Upstairs', 'Downstairs', 'Sitting',
       'Standing'], dtype=object)

In [5]:
window_size = 200
slide = 20
segments = []
labels = []
for i in range(0, len(df) - window_size, slide):
    xs = df['x-axis'].values[i: i+window_size]
    ys = df['y-axis'].values[i: i+window_size]
    zs = df['z-axis'].values[i: i+window_size]
    label = stats.mode(df['activity'][i: i+window_size])[0][0]
    segments.append([xs, ys, zs])
    labels.append(label)

/usr/local/lib/python3.5/dist-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [6]:
print('shape of segments =', np.array(segments).shape)
print('shape of labels =', np.array(labels).shape)
segments = np.asarray(segments, dtype=np.float32).reshape(-1, window_size, 3)
labels = np.asarray(pd.get_dummies(labels), dtype=np.float32)
print('After reshaping...')
print('shape of segments =', segments.shape)
print('shape of labels =', labels.shape)

shape of segments = (54901, 3, 200)
shape of labels = (54901,)
After reshaping...
shape of segments = (54901, 200, 3)
shape of labels = (54901, 6)


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(segments, labels, test_size=0.2, random_state=RANDOM_SEED)

In [8]:
print('X_train.shape =', X_train.shape)
print('Y_train.shape =', Y_train.shape)
print('X_test.shape =', X_test.shape)
print('Y_test.shape =', Y_test.shape)

X_train.shape = (43920, 200, 3)
Y_train.shape = (43920, 6)
X_test.shape = (10981, 200, 3)
Y_test.shape = (10981, 6)


In [9]:
Y_train[0]

array([0., 0., 0., 1., 0., 0.], dtype=float32)

In [10]:
n_channels = 3
n_labels = 6
n_hidden_units = 64

In [11]:
model = Sequential()

model.add(LSTM(10, return_sequences=True, input_shape=(window_size, n_channels))) 
model.add(LSTM(10, return_sequences=True))
model.add(LSTM(10))
model.add(Dense(n_labels, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200, 10)           560       
_________________________________________________________________
lstm_2 (LSTM)                (None, 200, 10)           840       
_________________________________________________________________
lstm_3 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 66        
Total params: 2,306
Trainable params: 2,306
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
model.fit(X_train, Y_train, batch_size=100, epochs=1, validation_data=(X_test, Y_test), verbose=1)

Train on 43920 samples, validate on 10981 samples
Epoch 1/1
43920/43920 [==============================] - 245s 6ms/step - loss: 0.9959 - acc: 0.6413 - val_loss: 0.7412 - val_acc: 0.7704

In [13]:
from keras.models import load_model

model.save('wisdm_lstm.h5')

In [14]:
del model

In [15]:
model = load_model('wisdm_lstm.h5')

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200, 10)           560       
_________________________________________________________________
lstm_2 (LSTM)                (None, 200, 10)           840       
_________________________________________________________________
lstm_3 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 66        
Total params: 2,306
Trainable params: 2,306
Non-trainable params: 0
_________________________________________________________________


In [24]:
predicted_y = np.argmax(model.predict(X_test), axis=1)

In [25]:
ground_truth_y = np.argmax(Y_test, axis=1)

In [30]:
accuracy = np.sum(predicted_y == ground_truth_y) / predicted_y.shape[0]

In [ ]:
print("Test Accuracy :", (accuracy*100), ""